Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину. 

Шаги для выбора локации обычно такие:

* В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
* Строят модель для предсказания объёма запасов в новых скважинах;
* Выбирают скважины с самыми высокими оценками значений;
* Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.

In [25]:
# импорт необходимых для проекта библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as st 
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [26]:
# Согласно условиям задачи
BUDGET = 10e9
PRICE_PER_UNIT = 450000
ALL_UNITS = 500
TOP_UNITS = 200
REQUIRED_VOLUME = BUDGET / PRICE_PER_UNIT / TOP_UNITS

# Загрузите и подготовьте данные. Поясните порядок действий.


In [27]:
# импорт файлов из предполагаемых директорий
pth1 = '/datasets/geo_data_0.csv'
pth2 = '/datasets/geo_data_1.csv'
pth3 = '/datasets/geo_data_2.csv'
pth4= '/content/drive/MyDrive/Colab Notebooks/datasets/geo_data_0.csv'
pth5 = '/content/drive/MyDrive/Colab Notebooks/datasets/geo_data_1.csv'
pth6 = '/content/drive/MyDrive/Colab Notebooks/datasets/geo_data_2.csv'

if os.path.exists(pth1):
    df1 = pd.read_csv(pth1)
    df2 = pd.read_csv(pth2)
    df3 = pd.read_csv(pth3)
elif os.path.exists(pth4):
    df1 = pd.read_csv(pth4)
    df2 = pd.read_csv(pth5)
    df3 = pd.read_csv(pth6)

else:
    print("Проверьте правильность пути к датасету")
# краткий обзор датасета
display(df1.info())
display(df2.info())
display(df3.info())
display(df1.describe())
display(df2.describe())
display(df3.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [28]:
# Проверям нет ли повторов в датасетах
for data in [df1,df2,df3]:
  print(data[data.duplicated()] == True)


Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []
Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []
Empty DataFrame
Columns: [id, f0, f1, f2, product]
Index: []


# Обучите и проверьте модель для каждого региона:



## 2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
## 2.2. Обучите модель и сделайте предсказания на валидационной выборке


In [29]:
def model(df):
    # Подготовливаем данные
    features = df.drop(['product', 'id'], axis=1)
    target = df['product']
    # Создаем тренировочную и валидационной выборки
    features_train, features_valid, target_train, target_valid= train_test_split(features, target, test_size=0.25, random_state=12345)
    display('Тренировочная выборка:', features_train.shape)
    display('Валидационная выборка:', features_valid.shape)
    # Масштабируем признаки
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    # Обучаем модель
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    predicted_valid = pd.Series(predicted_valid)
    # Расчет среднего запаса
    avg_predicted = predicted_valid.mean()
    print("Средний запас предсказанного сырья:", avg_predicted)
    # Расчет RMSE
    rmse = mean_squared_error(target_valid, predicted_valid)**0.5
    print("RMSE модели:", rmse)
    # Предскзаывание на константной модели
    const_model_predict = pd.Series(target_train.mean(), index=target_valid.index) 
    rmse = mean_squared_error(target_valid, const_model_predict) ** 0.5
    print("RMSE константной модели =", rmse)
  
    return predicted_valid.reset_index(drop=True), target_valid.reset_index(drop=True)



##  2.3. Сохраните предсказания и правильные ответы на валидационной выборке.
##  2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.


In [30]:
print('Первый регион')
predicted_valid_1, target_valid_1 = model(df1)
print()
print('Второй регион')
predicted_valid_2, target_valid_2 = model(df2)
print()
print('Третий регион')
predicted_valid_3, target_valid_3 = model(df3)

Первый регион


'Тренировочная выборка:'

(75000, 3)

'Валидационная выборка:'

(25000, 3)

...

'Тренировочная выборка:'

(75000, 3)

'Валидационная выборка:'

(25000, 3)

Средний запас предсказанного сырья: 94.96504596800489
RMSE модели: 40.02970873393434
RMSE константной модели = 44.90234968510566


 ## 2.5. Проанализируйте результаты.

RMSE моделей значительно больше константных моделей, что сообщает нам, что модель не обучилась достаточно хорошо и плохо выявляет закономерности в данных. Исключением является модель, обучавшаяся на данных **второго региона**.

# Подготовьтесь к расчёту прибыли:


## 3.1. Все ключевые значения для расчётов сохранены в отдельных переменных.


## 3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.


In [31]:

print('Достаточный объем сырья для безубыточной разработки новой скважины:', REQUIRED_VOLUME)
print('Средний запас в регионе 1:', df1['product'].mean())
print('Средний запас в регионе 2:', df2['product'].mean())
print('Средний запас в регионе 3:', df3['product'].mean())

Достаточный объем сырья для безубыточной разработки новой скважины: 111.11111111111111
Средний запас в регионе 1: 92.50000000000001
Средний запас в регионе 2: 68.82500000000002
Средний запас в регионе 3: 95.00000000000004


##  3.3. Напишите выводы по этапу подготовки расчёта прибыли.



Cредние запасы в регионах не соответствуют соответствуют рассчитаному уровню безубыточности.

# Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

Рассчитать прибыль для выбранных скважин с максимальными предсказаниями. 

In [32]:
def profit(target, predicted):
    #4.1 Выберите скважины с максимальными значениями предсказаний. 
    top_pred = predicted.sort_values(ascending=False)[:TOP_UNITS] # для 200
    # 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.   
    sum_target = target[top_pred.index].sum()
    # 4.3. Рассчитайте прибыль для полученного объёма сырья
    profit = sum_target * PRICE_PER_UNIT - BUDGET
    return profit

print('Прибыль для региона 1:', profit(target_valid_1, predicted_valid_1))
print('Прибыль для региона 2:', profit(target_valid_2, predicted_valid_2))
print('Прибыль для региона 3:', profit(target_valid_3, predicted_valid_3))

Прибыль для региона 1: 3320826043.1398506
Прибыль для региона 2: 2415086696.681511
Прибыль для региона 3: 2710349963.5998325


# 5. Посчитайте риски и прибыль для каждого региона:


## 5.1. + 5.2. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [33]:
state = np.random.RandomState(42)

def find_conf_interval_and_loss_risk(predicted_valid, target_valid):
    # 5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    # Cоздаем списк с 1000 значение - распределение прибыли
    revenue = []
    for i in range(1000):
        target_sample = target_valid.sample(ALL_UNITS, replace=True, random_state=state)
        predicted_sample = predicted_valid[target_sample.index]
        revenue.append(profit(target_sample, predicted_sample))

    revenue = pd.Series(revenue)

    # 5.2 Найдите среднюю прибыль,
    mean_revenue = revenue.mean()
    # Определение границ 95%-го доверительного интервала,
    lower = revenue.quantile(0.025)
    higher = revenue.quantile(0.975)
    # Риск убытков: как сумма убыточных выборок к общему числу выборок.
    risk = (revenue < 0).sum() / 1000 

    return ((lower, higher), mean_revenue, risk)


table = []
i = 1
for preds, target in zip([predicted_valid_1, predicted_valid_2, predicted_valid_3], 
                         [target_valid_1, target_valid_2, target_valid_3]):
    interval, mean_revenue, risk = find_conf_interval_and_loss_risk(preds, target)
    row = {'Регион': i, 
           'Средняя прибыль': mean_revenue, 
           '95-% доверительный интервал': interval, 
           'Риск убытка': risk}
    table.append(row)
    i+=1
pd.DataFrame(table)

,Регион,Средняя прибыль,95-% доверительный интервал,Риск убытка
0,1,5.867268e+08,"(-23098498.171101615, 1189482667.2667706)",0.027
1,2,6.626946e+08,"(141539735.44347674, 1208722487.7387738)",0.007
2,3,6.217730e+08,"(5046868.11313715, 1243004614.896015)",0.021


## 5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

Где бурить новую скважину?

**Второй регион** показывает самый узкий доверительный интервал, а также минимальный риск убытка. Предполагаемая средняя прибыль в этом регионе так же выше остальных
